In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import SwinConfig, SwinModel
from tqdm import tqdm
import matplotlib.pyplot as plt

# --- Cấu hình chung ---
IMG_SIZE = 256
BATCH_SIZE = 12
LEARNING_RATE = 1e-4
NUM_EPOCHS = 1000 # Số epoch bạn muốn huấn luyện
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# --- Định nghĩa Mô hình (Tái sử dụng từ mã của bạn, với một số điều chỉnh) ---

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        # Sử dụng nn.Upsample + Conv2d(1x1) để tránh checkerboard artifacts
        self.upsample = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1)
        )
        
        # ConvBlock sẽ nhận đầu vào từ upsample và skip_features (nếu có)
        self.conv_block = ConvBlock(out_channels + skip_channels, out_channels)

    def forward(self, x, skip_features=None):
        x = self.upsample(x)
        
        if skip_features is not None:
            # Đảm bảo kích thước không gian của skip_features khớp với x
            if x.shape[2:] != skip_features.shape[2:]:
                skip_features = F.interpolate(skip_features, size=x.shape[2:], mode='bilinear', align_corners=True)
            x = torch.cat([x, skip_features], dim=1)
        
        x = self.conv_block(x)
        return x

class SwinUNet(nn.Module):
    class SwinUNet(nn.Module):
    def __init__(self, input_channels=3, num_classes=1):
        super().__init__()
        self.IMG_SIZE = IMG_SIZE

        config = SwinConfig(image_size=self.IMG_SIZE, num_channels=input_channels,
                            patch_size=4, embed_dim=128, depths=[2, 2, 18, 2], num_heads=[4, 8, 16, 32],
                            window_size=7, mlp_ratio=4., qkv_bias=True, hidden_dropout_prob=0.0,
                            attention_probs_dropout_prob=0.0, drop_path_rate=0.1,
                            hidden_act="gelu", use_absolute_embeddings=False,
                            patch_norm=True, initializer_range=0.02, layer_norm_eps=1e-05,
                            out_features=["stage1", "stage2", "stage3", "stage4"])
        self.swin = SwinModel(config)

        # Bottleneck: từ hidden_states[3] (H/32 x W/32)
        # config.embed_dim * 8 = 128 * 8 = 1024
        self.bottleneck = ConvBlock(config.embed_dim * 8, config.embed_dim * 8)

        # Decoder 4: upsample từ H/32 -> H/16. Skip từ hidden_states[2] (H/16)
        # in_channels: từ bottleneck (1024), skip_channels: từ hidden_states[2] (512), out_channels: 512
        self.decoder4 = DecoderBlock(in_channels=config.embed_dim * 8, skip_channels=config.embed_dim * 4, out_channels=config.embed_dim * 4)

        # Decoder 3: upsample từ H/16 -> H/8. Skip từ hidden_states[1] (H/8)
        # in_channels: từ decoder4 (512), skip_channels: từ hidden_states[1] (256), out_channels: 256
        self.decoder3 = DecoderBlock(in_channels=config.embed_dim * 4, skip_channels=config.embed_dim * 2, out_channels=config.embed_dim * 2)

        # Decoder 2: upsample từ H/8 -> H/4. Skip từ hidden_states[0] (H/4)
        # in_channels: từ decoder3 (256), skip_channels: từ hidden_states[0] (128), out_channels: 128
        self.decoder2 = DecoderBlock(in_channels=config.embed_dim * 2, skip_channels=config.embed_dim * 1, out_channels=config.embed_dim * 1)

        # Decoder 1: upsample từ H/4 -> H/2. Không có skip trực tiếp từ Swin Encoder ở cấp độ này
        # in_channels: từ decoder2 (128), skip_channels: 0, out_channels: 64
        self.decoder1 = DecoderBlock(in_channels=config.embed_dim * 1, skip_channels=0, out_channels=config.embed_dim // 2)

        # Final Upsample: upsample từ H/2 -> H. Không có skip
        # in_channels: từ decoder1 (64), skip_channels: 0, out_channels: 32
        self.final_upsample = DecoderBlock(in_channels=config.embed_dim // 2, skip_channels=0, out_channels=config.embed_dim // 4)

        # Final convolution
        self.final_conv = nn.Conv2d(config.embed_dim // 4, num_classes, kernel_size=1)

    def forward(self, x):
        outputs = self.swin(pixel_values=x, output_hidden_states=True)
        
        # Lấy các hidden_states tương ứng với các cấp độ giải mã
        # hidden_states[0]: H/4 x W/4, 96 kênh
        # hidden_states[1]: H/8 x W/8, 192 kênh
        # hidden_states[2]: H/16 x W/16, 384 kênh
        # hidden_states[3]: H/32 x W/32, 768 kênh (thường dùng cho bottleneck)
        # hidden_states[4]: Đây là output của Stage 3 trước Layernorm (giống hidden_states[3])
        
        # Chuyển đổi các hidden_states từ format (batch, num_patches, embed_dim)
        # sang (batch, embed_dim, height, width)
        
        # Skip connection cho Decoder2 (H/4)
        hs0 = outputs.hidden_states[0]
        batch_size, num_patches, embed_dim = hs0.shape
        side = int(np.sqrt(num_patches)) # side = IMG_SIZE / 4 = 256 / 4 = 64
        skip0 = hs0.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side) # (B, 96, 64, 64)

        # Skip connection cho Decoder3 (H/8)
        hs1 = outputs.hidden_states[1]
        batch_size, num_patches, embed_dim = hs1.shape
        side = int(np.sqrt(num_patches)) # side = IMG_SIZE / 8 = 32
        skip1 = hs1.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side) # (B, 192, 32, 32)

        # Skip connection cho Decoder4 (H/16)
        hs2 = outputs.hidden_states[2]
        batch_size, num_patches, embed_dim = hs2.shape
        side = int(np.sqrt(num_patches)) # side = IMG_SIZE / 16 = 16
        skip2 = hs2.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side) # (B, 384, 16, 16)

        # Bottleneck từ hidden_states[3] hoặc [4]. [4] thường là output cuối cùng của encoder
        # mà bạn đã dùng trong mã inference, nên tôi sẽ giữ nguyên.
        x_bottleneck = outputs.hidden_states[4]
        batch_size, num_patches, embed_dim = x_bottleneck.shape
        side = int(np.sqrt(num_patches)) # side = IMG_SIZE / 32 = 8
        x_bottleneck = x_bottleneck.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side) # (B, 768, 8, 8)
        
        x = self.bottleneck(x_bottleneck) # (B, 768, 8, 8)

        # Giải mã
        x = self.decoder4(x, skip2) # in (768, 8, 8), skip (384, 16, 16) -> out (384, 16, 16)
        x = self.decoder3(x, skip1) # in (384, 16, 16), skip (192, 32, 32) -> out (192, 32, 32)
        x = self.decoder2(x, skip0) # in (192, 32, 32), skip (96, 64, 64) -> out (96, 64, 64)
        x = self.decoder1(x)       # in (96, 64, 64), no skip           -> out (48, 128, 128)
        x = self.final_upsample(x) # in (48, 128, 128), no skip         -> out (24, 256, 256)

        outputs = self.final_conv(x) # (B, 1, 256, 256)
        # Không áp dụng sigmoid ở đây. BCEWithLogitsLoss sẽ làm điều đó.
        # Nếu bạn muốn dự đoán đầu ra 0-1, bạn có thể áp dụng sigmoid sau khi loss được tính.
        # outputs = self.sigmoid(outputs) 

        return outputs

# --- Định nghĩa Hàm Mất Mát (Loss Functions) ---

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        # Flatten inputs and targets
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        # Calculate intersection and union
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + self.smooth) / (inputs.sum() + targets.sum() + self.smooth)
        
        return 1 - dice # Return (1 - Dice Coefficient)

class CombinedLoss(nn.Module):
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce_loss = nn.BCEWithLogitsLoss() # Sử dụng cho logits (output chưa sigmoid)
        self.dice_loss = DiceLoss()

    def forward(self, predictions, targets):
        # BCEWithLogitsLoss nhận logits và targets. Targets phải là float.
        bce = self.bce_loss(predictions, targets)
        
        # Để tính Dice Loss, chúng ta cần chuyển predictions thành xác suất (0-1)
        # bằng sigmoid, vì Dice hoạt động tốt nhất trên xác suất hoặc nhị phân.
        predictions_sigmoid = torch.sigmoid(predictions)
        dice = self.dice_loss(predictions_sigmoid, targets)
        
        return self.bce_weight * bce + self.dice_weight * dice

# --- Giả lập Dataset (thay thế bằng Dataset thực của bạn) ---
class DummySegmentationDataset(Dataset):
    def __init__(self, num_samples=100, img_size=IMG_SIZE):
        self.num_samples = num_samples
        self.img_size = img_size
        print(f"Khởi tạo DummyDataset với {num_samples} mẫu.")

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Tạo ảnh giả (ngẫu nhiên)
        image = np.random.rand(self.img_size, self.img_size, 3).astype(np.float32)
        # Tạo mask giả (ngẫu nhiên nhị phân)
        mask = (np.random.rand(self.img_size, self.img_size) > 0.5).astype(np.float32)

        # Chuyển đổi sang định dạng PyTorch (C, H, W)
        image_tensor = torch.from_numpy(image).permute(2, 0, 1)
        mask_tensor = torch.from_numpy(mask).unsqueeze(0) # Thêm chiều kênh

        return image_tensor, mask_tensor

# --- Tiền xử lý ảnh (giống như trong file inference) ---
# Hàm này có thể được tích hợp vào Dataset của bạn để xử lý ảnh thực
def load_and_preprocess_image_and_mask(image_path, mask_path, target_size=IMG_SIZE, window_size=4):
    img = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) # Đọc mask dưới dạng ảnh grayscale

    if img is None or mask is None:
        raise ValueError(f"Không thể đọc tệp ảnh hoặc mask: {image_path}, {mask_path}")
    
    h, w = img.shape[:2]
    new_h = max(target_size, ((h + window_size - 1) // window_size) * window_size)
    new_w = max(target_size, ((w + window_size - 1) // window_size) * window_size)

    # Padding ảnh
    padded_img = np.zeros((new_h, new_w, 3), dtype=img.dtype)
    padded_img[:h, :w, :] = img
    cropped_img = padded_img[:target_size, :target_size, :]

    # Padding mask
    padded_mask = np.zeros((new_h, new_w), dtype=mask.dtype)
    padded_mask[:h, :w] = mask
    cropped_mask = padded_mask[:target_size, :target_size]

    img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    img_processed = img_rgb.astype(np.float32) / 255.0
    
    # Chuẩn hóa mask về 0-1 và đảm bảo là float
    mask_processed = (cropped_mask > 127).astype(np.float32) # Giả sử mask nhị phân (0 hoặc 255)

    img_tensor = torch.from_numpy(img_processed).permute(2, 0, 1)
    mask_tensor = torch.from_numpy(mask_processed).unsqueeze(0) # Thêm chiều kênh
    
    return img_tensor, mask_tensor

# --- Dataset thực tế (ví dụ cho cấu trúc thư mục) ---
class RealSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, img_size=IMG_SIZE):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.img_size = img_size
        
        self.image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.mask_filenames = sorted([f for f in os.listdir(mask_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

        # Đảm bảo số lượng ảnh và mask khớp nhau
        if len(self.image_filenames) != len(self.mask_filenames):
            print(f"Cảnh báo: Số lượng ảnh ({len(self.image_filenames)}) và mask ({len(self.mask_filenames)}) không khớp.")
            # Bạn có thể thêm logic để lọc ra các cặp không khớp hoặc báo lỗi.

        print(f"Tìm thấy {len(self.image_filenames)} cặp ảnh/mask trong thư mục.")

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        mask_name = self.mask_filenames[idx] # Giả định tên mask tương ứng với tên ảnh

        image_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name) # Đây là giả định quan trọng, cần đảm bảo tên mask khớp

        image, mask = load_and_preprocess_image_and_mask(image_path, mask_path, self.img_size)
        return image, mask

# --- Chỉ số đánh giá (Metrics) ---
def dice_coefficient(predictions, targets, smooth=1e-6):
    # Áp dụng sigmoid và làm tròn để có mask nhị phân cho đánh giá
    predictions = (torch.sigmoid(predictions) > 0.5).float()
    
    # Flatten tensors
    predictions = predictions.view(-1)
    targets = targets.view(-1)
    
    intersection = (predictions * targets).sum()
    union = predictions.sum() + targets.sum()
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

def iou_score(predictions, targets, smooth=1e-6):
    predictions = (torch.sigmoid(predictions) > 0.5).float()

    predictions = predictions.view(-1)
    targets = targets.view(-1)

    intersection = (predictions * targets).sum()
    total = predictions.sum() + targets.sum()
    union = total - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    return iou

# --- Hàm Huấn luyện ---
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, checkpoint_dir):
    best_val_iou = 0.0
    history = {'train_loss': [], 'val_loss': [], 'val_dice': [], 'val_iou': []}

    for epoch in range(num_epochs):
        model.train() # Chế độ huấn luyện
        running_loss = 0.0
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for images, masks in train_bar:
            images = images.to(device)
            masks = masks.to(device) # Mask phải là float (0.0 hoặc 1.0)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        # --- Đánh giá trên tập Validation ---
        model.eval() # Chế độ đánh giá
        val_loss = 0.0
        val_dice_scores = []
        val_iou_scores = []
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")

        with torch.no_grad(): # Không tính gradient trong quá trình đánh giá
            for images, masks in val_bar:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

                dice = dice_coefficient(outputs, masks)
                iou = iou_score(outputs, masks)
                val_dice_scores.append(dice.item())
                val_iou_scores.append(iou.item())
                val_bar.set_postfix(val_loss=loss.item(), dice=dice.item(), iou=iou.item())

        avg_val_loss = val_loss / len(val_loader)
        avg_val_dice = np.mean(val_dice_scores)
        avg_val_iou = np.mean(val_iou_scores)
        
        history['val_loss'].append(avg_val_loss)
        history['val_dice'].append(avg_val_dice)
        history['val_iou'].append(avg_val_iou)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, Val Dice: {avg_val_dice:.4f}, Val IoU: {avg_val_iou:.4f}")

        # Lưu mô hình tốt nhất dựa trên IoU
        if avg_val_iou > best_val_iou:
            best_val_iou = avg_val_iou
            checkpoint_path = os.path.join(checkpoint_dir, 'swin_unet_best_pytorch.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_val_loss,
                'iou': avg_val_iou,
                'dice': avg_val_dice,
            }, checkpoint_path)
            print(f"Đã lưu mô hình tốt nhất với Val IoU: {best_val_iou:.4f} tại {checkpoint_path}")
    
    return history

# --- Khởi tạo và Bắt đầu Huấn luyện ---
if __name__ == '__main__':
    print(f"Thiết bị đang sử dụng: {DEVICE}")

    # --- Khởi tạo Dataset và DataLoader ---
    # THAY THẾ DUMMY DATASET BẰNG REAL DATASET CỦA BẠN
    # Ví dụ với cấu trúc thư mục như UDTIRI-Crack bạn nhắc đến:
    # UDTIRI-Crack/UDTIRI-Crack Generalization2/image/
    # UDTIRI-Crack/UDTIRI-Crack Generalization2/mask/

    # Hãy đảm bảo rằng tên tệp ảnh và mask khớp nhau, ví dụ:
    # image/019.png -> mask/019.png

    # Dùng DummyDataset để test code nhanh nếu chưa có dữ liệu sẵn
    # train_dataset = DummySegmentationDataset(num_samples=100)
    # val_dataset = DummySegmentationDataset(num_samples=20)
    
    # Ví dụ sử dụng RealSegmentationDataset (thay đổi đường dẫn phù hợp)
    try:
        # Thay đổi đường dẫn này cho phù hợp với máy tính của bạn
        base_data_path = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\archive (2)\UDTIRI-Crack\UDTIRI-Crack Generalization2'
        train_image_dir = os.path.join(base_data_path, 'image')
        train_mask_dir = os.path.join(base_data_path, 'mask')

        # Nếu bạn có tập validation riêng, hãy tạo các thư mục tương ứng
        # val_image_dir = os.path.join(base_data_path, 'val_images')
        # val_mask_dir = os.path.join(base_data_path, 'val_masks')

        # Tạm thời sử dụng cùng một thư mục cho cả train/val để demo, NHƯNG KHÔNG NÊN LÀM VẬY TRONG THỰC TẾ
        # Bạn nên chia dữ liệu thành train/val/test một cách hợp lý.
        full_dataset = RealSegmentationDataset(image_dir=train_image_dir, mask_dir=train_mask_dir)
        
        # Chia dataset thành train và validation
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

        print(f"Kích thước tập huấn luyện: {len(train_dataset)} mẫu")
        print(f"Kích thước tập validation: {len(val_dataset)} mẫu")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=os.cpu_count() // 2 or 1)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=os.cpu_count() // 2 or 1)

    except Exception as e:
        print(f"Lỗi khi tải dữ liệu thực: {e}. Sẽ dùng DummyDataset để tiếp tục.")
        train_dataset = DummySegmentationDataset(num_samples=100)
        val_dataset = DummySegmentationDataset(num_samples=20)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


    # --- Khởi tạo Mô hình, Hàm mất mát, Bộ tối ưu hóa ---
    model = SwinUNet(input_channels=3, num_classes=1).to(DEVICE)
    criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.5) # Có thể điều chỉnh trọng số
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("Bắt đầu huấn luyện mô hình...")
    training_history = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE, CHECKPOINT_DIR)
    print("Huấn luyện hoàn tất!")

    # --- Vẽ biểu đồ lịch sử huấn luyện ---
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(training_history['train_loss'], label='Train Loss')
    plt.plot(training_history['val_loss'], label='Val Loss')
    plt.title('Loss theo Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(training_history['val_dice'], label='Val Dice Coefficient')
    plt.plot(training_history['val_iou'], label='Val IoU Score')
    plt.title('Metrics theo Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

IndentationError: expected an indented block after class definition on line 63 (318673341.py, line 64)